    - Onderstaande query af en toe draaien om te checken op nieuwe wedkantoren vanuit bron ODDAPI

In [1]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal alle wedkantoren in oddapi op, én bestaande wedkantoren in d_wedkantoren
wedkantoren_oddapi = pd.read_sql('SELECT distinct(wedkantoor) FROM dsa.oddapi_odds;', sqal_connection)
bestaande_wedkantoren = pd.read_sql("SELECT * FROM dwa.d_wedkantoren ", sqal_connection)

# WEDKANTOREN ODDAPI
# Check of wedkantoren uit oddapi voorkomen in bestaande_wedkantoren
# Zo niet dan voer je dit wedkantoor in
# Invoer komt in lijst te staan
print(bestaande_wedkantoren)
wedkantoor_oddapi_lijst = []
for i, wedkantoor_oddapi in wedkantoren_oddapi.iterrows():
    if wedkantoor_oddapi['wedkantoor'] not in bestaande_wedkantoren['wk_oddapi'].tolist():
        wedkantoor = wedkantoor_oddapi['wedkantoor']
        wedkantoorsleutel = input(f'wedkantoorsleutel voor wk_oddapi {wedkantoor}').lower()
        beschikbaar_nl = input(f'beschikbaar_nl True/False voor wk_oddapi {wedkantoor}')
        if beschikbaar_nl.lower() == 'true':
            beschikbaar_nl = bool('true')
        else:
            beschikbaar_nl = bool('')
        actief = input(f'actief True/False voor wk_oddapi {wedkantoor}')
        if actief.lower() == 'true':
            actief = bool('true')
        else:
            actief = bool('')
        wedkantoor_oddapi_lijst.append((wedkantoorsleutel, wedkantoor, beschikbaar_nl, actief))

# Pak de lijst met nieuw ingevoerde wedkantoren voor oddapi
# Als dit wedkantoor al voorkomt vanuit fd_stats dan UPDATE
# Ander nieuw record toevoegen
for i in wedkantoor_oddapi_lijst:
    if i[0] in bestaande_wedkantoren['wedkantoor'].tolist():
        psy_cursor.execute("""
                    UPDATE dwa.d_wedkantoren
                    SET wk_oddapi = %s, beschikbaar_nl = %s, actief = %s
                    WHERE wedkantoor = %s;                      
                    """,
                    (i[1], i[2], i[3], i[0]))
    else: 
         psy_cursor.execute("""
                    INSERT INTO dwa.d_wedkantoren (wedkantoor, wk_fd, wk_oddapi, beschikbaar_nl, actief)
                    VALUES (%s, %s, %s, %s, %s);                        
                    """
                    ,(i[0], None, i[1], i[2], i[3]))

# update upate_tabel
psy_cursor.execute("""
                        UPDATE dsa.update
                        SET update_datum = current_date
                        WHERE tabelnaam = 'd_wedkantoren';                      
                        """)

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close()            

      wedkantoor wk_fd        wk_oddapi  beschikbaar_nl  actief
0         bet365  B365             None           False    True
1    blue square    BS             None           False    True
2        bet&win    BW             None           False    True
3    gamebookers    GB             None           False    True
4    interwetten    IW             None           False    True
5      ladbrokes    LB             None           False    True
6       sporting    SO             None           False    True
7    sportingbet    SB             None           False    True
8     stan james    SJ             None           False    True
9     stanleybet    SY             None           False    True
10        vc bet    VC             None           False    True
11     matchbook  None        matchbook           False    True
12       betfair  None    betfair_ex_eu           False    True
13       betclic  None          betclic           False    True
14     everygame  None        everygame 

    Hieronder eenmalige invoer van wedkantoren uit bron football stats

In [58]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Voer wedkantoren van fd_stats eenmalig in
wedkantoren_fd = (['B365', 'bet365'], ['BS', 'blue square'], ['BW', 'bet&win'], ['GB', 'gamebookers'],
                    ['IW', 'interwetten'], ['LB', 'ladbrokes'], ['PS', 'pinnacle'], ['SO', 'sporting'], 
                    ['SB', 'sportingbet'], ['SJ', 'stan james'], ['SY', 'stanleybet'],
                    ['VC', 'vc bet'], ['WH', 'william hill'])

for wedkantoor in wedkantoren_fd:
    psy_cursor.execute("""
                        INSERT INTO dwa.d_wedkantoren (wedkantoor, wk_fd, wk_oddapi, beschikbaar_nl, actief)
                        VALUES (%s, %s, %s, %s, %s);                        
                        """
                        ,(wedkantoor[1], wedkantoor[0], None, False, True))

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close()